# Tabulation of Fractional Cover data within tiled shapefile polygons

**What does this notebook do?**

This notebook is a pilot collaboration between Geoscience Australia and Australian Bureau of Statistics. The purpose of the notebook is to use a shapefile of SA2 boundaries to load the Fractional Cover (FC) dataset, complete zonal statistics and tabulate the results.

This workflow has been created to run large SA2 within memory on the VDI and uses pre-tiled SA2 polygons as an input. Once the monthly tabulation script is complete the workflow reads the tabulated results and calculates annual results and quality metrics. 

**Requirements**

A shapefile that contains **tiled** SA2 polygons. The Intersect tool in ArcGIS was used to break the large (>20 000 km2) SA2 into Australian Albers 100 km by 100 km tiles.

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles`

`module load dea/20181213`


**Background**

Data from the Landsat 5,7 and 8 satellite missions are accessible through Digital Earth Australia (DEA). The code snippets in this notebook will let you retrieve and tabulate the FC data stored in DEA.


**How to use this notebook**

A basic understanding of any programming language is desirable but one doesn't have to be an expert Python programmer to manipulate the code to get and display the data.This doc applies to the following Landsat satellites, Fractional Cover bands and the WOfS dataset:

- Landsat 5
- Landsat 7
- Landsat 8
- PV - Photosythetic vegetation
- NPV - Non-Photosythetic vegetation
- BS - Bare Soil
- UE - Unmixing Error
- Water Observations from Space (WOFs)
- WOfS Feature Layer (WOFL)

**Bugs still to fix**
-  NA

**Errors or bugs**

If you find an error or bug in this notebook, please contact erin.telfer@ga.gov.au.


## 1. Import Libraries

In [14]:
%matplotlib inline

from datetime import time, datetime
import os.path

from matplotlib import pyplot as plt
import pandas as pd
import numpy
import csv
import xarray as xr
import rasterio
import rasterio.features
import fiona
from datetime import datetime
import dask
from dask.delayed import delayed
from dask.distributed import LocalCluster, Client
import tempfile

import datacube
from datacube import Datacube
from datacube.virtual import construct, construct_from_yaml
from datacube.ui.task_app import year_splitter
from datacube.utils.geometry import CRS

In [61]:
#input years range of interest
start_year = 2015
end_year = 2017

#input batch details
initials = 'ET'
batch_sa2_size = 'over20000_702051066'
run_date = '11June19'
run_notes = 'rerun of the last batch to include SA2 of Gulf'

## 2. USER INPUT

## 3. Set directory and shapefile details

In [62]:
#Set folder location and shapefile name
shapefile_path = '/g/data/r78/ext547/abs/input/SA_2016_clipped_albers_3577.shp'
#Set output folder
output_path = '/g/data/r78/ext547/abs/output/'
#Set name for output csv
name_of_output_file =f'tabulate_FC_{run_date}_{batch_sa2_size}_{start_year}_{end_year}'

In [63]:
#Years of interest are saved in expected format
time_range = (str(start_year), str(end_year))
print(time_range)

('2015', '2017')


## 4. Set up a local dask cluster
Some calculations take more memory than is available on a system.  By breaking the data up into chunks, we can chain a sequence of operations together, and work on the data a small piece at a time.

This lets several processes work at the same time, and manage total memory usage for the calculations.

In more advanced setups, we can distribute the work across multiple computers, using all of their memory and CPU power.

* We set `n_workers` to be the number of worker applications we want to run in the background, doing the processing of the chunk-based steps we have chained together.
* The `mem_per_worker` setting defines how much memory at most is available to each of the workers.
* `chunk_size` sets the width and height of the chunk in pixels of the size will break up the data into.

VDI has 8 CPUs available, and a total of 32GB of memory, but you will typically be sharing those with several (2-10) other users.   

In [64]:
#Set up dask cluster
n_workers = 7
threads_per_worker=1
mem_per_worker = 8e9  # 8e9 is 8GB (8,000,000,000 bytes)

chunk_size = {'time': 1, 'x': 2000, 'y': 2000}

In [66]:
cluster = LocalCluster(local_dir=tempfile.gettempdir(), 
                       n_workers=n_workers, 
                       threads_per_worker=threads_per_worker,
                       memory_limit=mem_per_worker)
client = Client(cluster)
dask.config.set(get=client.get)
client

Client Scheduler: tcp://127.0.0.1:33090 Dashboard: http://127.0.0.1:33612/status,Cluster Workers: 7 Cores: 7 Memory: 56.00 GB


We also get a dashboard to see how the system is running, by clicking the link above after the cell has been run.

## 5. Connect to the Datacube 

In [67]:
dc = Datacube()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-06-11 20:59:53


## 6. Construct the virtual product

In [68]:
#Remove Landsat 7 scenes with the Scan Line Correction (SLC) missing data
LS7_BROKEN_DATE = datetime(2003, 5, 31)
is_pre_slc_failure = lambda dataset: dataset.center_time < LS7_BROKEN_DATE
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-06-11 20:59:53


In [69]:
#Create function to ensure wofls in correct format
def wofls_fuser(dest, src):
    where_nodata = (src & 1) == 0
    numpy.copyto(dest, src, where=where_nodata)
    return dest
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-06-11 20:59:53


In [70]:
#Create virtual product so that datacube data can be loaded effectively within memory
fc_and_water_yaml = """
        juxtapose:
          - collate:
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls5_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls5_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls7_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                        # dataset_predicate: __main__.is_pre_slc_failure
                      - transform: make_mask
                        input:
                            product: ls7_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls8_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls8_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
          - transform: make_mask
            input:
                product: wofs_albers
                group_by: solar_day
                fuse_func: __main__.wofls_fuser
            flags:
                wet: true
            mask_measurement_name: water
"""
fc_and_water = construct_from_yaml(fc_and_water_yaml)

## 7. Set up functions

In [71]:
def geometry_mask(geoms, geobox, all_touched=False, invert=False, chunks=None):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    data = rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
    if chunks is not None:
        data = dask.array.from_array(data, chunks=tuple(chunks[d] for d in geobox.dims))
        
    coords = [xr.DataArray(data=coord.values, name=dim, dims=[dim], attrs={'units': coord.units}) 
              for dim, coord in geobox.coords.items()]
    return xr.DataArray(data, coords=coords)
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-06-11 20:59:53


In [72]:
def get_shapes(shape_file):
    """
    Extract spatial inforamtion from polygons within shapefile
    """
    with fiona.open(shape_file) as shapes:
        crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
        for shape in shapes:
            if shape['geometry'] is None:
                continue
            geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
            geom = geom.to_crs(CRS('EPSG:3577'))
            yield geom, shape['properties']

In [73]:
def fc_and_water_summary(data, mask_int):
    """
    Calculate the percentage and the area of each FC component, 
    water, and unclassified data within each SA2 boundary. 
    """
    # Where there is water, set the FC bands to 0%
    valid_data = numpy.isfinite(data['BS'])
    fc = data[['PV', 'NPV', 'BS']].where(data.water==0, other=0)
    fc['water'] = data.water.where(valid_data) * numpy.float32(100)
    fc = fc.apply(lambda data_array: data_array.clip(0, 100).where(valid_data))

    has_data = valid_data.groupby(valid_data.time.astype('datetime64[M]'), squeeze=False).sum(dim='time', skipna=False)
    has_data = has_data.sum(dim=['x','y'], skipna=True) #/ (mask_int / 100)

    # Flatten to a monthly mean
    fc = fc.groupby(fc.time.astype('datetime64[M]'), squeeze=False).mean(dim='time', skipna=True)
    # Calculate percentage of cover based on area of mask
    percentage = fc.sum(dim=['x','y']) * (100 / mask_int)
    for da in percentage.data_vars.values():
        da.attrs['units'] = '%'
    fc['mask_int'] = mask_int         
    #calculate area of fc components
    pixel_area_in_metres2 = 25 * 25
    m2_to_km2 = (1 / 1_000_000)
    percent_to_fraction = (1 / 100)
    area = (fc * (pixel_area_in_metres2 * m2_to_km2 * percent_to_fraction)).sum(dim=['x','y'])
    area = area.rename({'BS': 'BS_area', 
                        'PV': 'PV_area', 
                        'NPV': 'NPV_area', 
                        'water': 'water_area',
                        'mask_int':'mask_area',})

    for da in area.data_vars.values():
        da.attrs['units'] = 'km^2'
    fc = percentage.merge(area)
    fc['average_data_count'] = has_data
    fc['mask_int'] = mask_int

    return fc

In [74]:
def month_splitter(start_year, end_year_inclusive):
    """ 
    Split specified years into months 
    """
    yield from (str(p) for p in pd.period_range(start=pd.Period(start_year).start_time, 
                               end=pd.Period(end_year_inclusive).end_time, 
                               freq='M'))

In [75]:
def output_csv(input_ds, sa2_id, sa2_name, sa2_size, albers_id,name_of_output_file, monthly_or_annual='frequency'):
    """
    Save tabulated data into a csv
    """
    input_ds = input_ds.to_dataframe()
    input_ds.insert(0,'SA2_id', sa2_id)
    input_ds.insert(1,'SA2_name', sa2_name)
    input_ds.insert(2,'albers_id', albers_id)
    input_ds.insert(3,'SA2_size', sa2_size)
    input_ds.to_csv(f"{output_path}/{name_of_output_file}_{monthly_or_annual}.csv",mode='a',header=False)  

## 8. Set up the query
For each year and polygon query the product, apply the geometry mask and compute the fractional cover stats

Using `client.compute()` lets us use the monthly results in calculating the annual results at the same time.

In [76]:
#Obtain spatial information from shapefile
shape_file = os.path.expanduser(f'{shapefile_path}')
shapes = list(get_shapes(shape_file))

#Specify a particular SA2 boundary, if required, otherwise keep hashed out
shapes = [(g,p) for g, p in shapes if str(p['SA2_MAIN16']) == '702051066']
# shapes = [(g,p) for g, p in shapes if int(p['AREASQKM16']) < 25_000]

print(len(shapes))

19


If we have enough resources, we can start the query and calculation of the next year's data while the previous is still being calculated. `by_slice=False` will be faster, but use more memory.

For larger areas `by_slice` will need to be `True`, so that the compute cluster does not become overwhelmed.  

If you get the error:
> `distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting`

then you will need to set `by_slice=True`

In [77]:
by_slice=True

In [78]:
monthly_values = None
def process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, name_of_output_file, time_range):
    """ loop through each month within the time range and load fc data with virtual products, 
    mask to tiled SA2 boundary, tabulate monthly results and append to output.
    """
    monthly_values = []
    annual_values = []
    mask = None
    
    # Set geometry values to use Virtual Products, use this instead of `geopolygon=geometry` 
    search_terms = {
        'x': (geometry.envelope.left, geometry.envelope.right),
        'y': (geometry.envelope.top, geometry.envelope.bottom),
        'crs': str(geometry.crs), }  

    #loop through months within the date range
    for sub_time_range in month_splitter(time_range[0], time_range[-1]):
        print(sub_time_range)
        
        #load fc data using Virtual Products
        try:
            data = fc_and_water.load(dc, dask_chunks=chunk_size, 
                                     time=sub_time_range, 
                                     **search_terms)
        #if there is no data within this month, skip the load, and continue a load with the next month
        except ValueError:
            print(f'    No data for {sub_time_range} , skipping...')
            continue
        
        #mask data with tiled SA2 polygon to remove data that is outside of the polygon
        if mask is None:
            mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
            mask_int = mask * 1 #turn variable into an int
            mask_int = mask_int.sum() * 100 #create a varaible that counts the number of pixels within the tiles SA2 polygon
        data = data.where(mask) 
        
        #use function to tabulate fc components
        monthly_data = fc_and_water_summary(data, mask_int)
        #load data into memory from dask
        monthly_data = client.compute(monthly_data, sync=by_slice)
        #append monthly results into a list that contains the values for all other months 
        monthly_values.append(monthly_data)
        
    if not by_slice:
        print("  all years queried, hard load data")
        monthly_values = client.gather(monthly_values)
    #concatenate the monthly results together into an xarray    
    monthly_values = xr.concat(monthly_values, dim='time')
    
    #append results to a csv
    print("All data loaded, save to csv")
    output_csv(monthly_values, sa2_id, sa2_name, sa2_size,albers_id, name_of_output_file, monthly_or_annual='monthly')
       
    print(f"SA2 {sa2_id} done")

## 9. Create empty CSVs to save results

In [80]:
#Specified headings
header = ['DATE','SA2_MAIN16', 'SA2_NAME16', 'ALBERS3577_ID', 'AREASQKM16',
          'PV_PERCENTAGE','NPV_PERCENTAGE','BS_PERCENTAGE','WOFL_PERCENTAGE',
          'PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
          'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','FC_AREA_SQKM_ALBERS3577', 
          'AVERAGE_OBSERVATION_COUNT','SA2_TILE_PIXEL_COUNT']

error_header = ['','RUN_NAME','SA2_MAIN16','DATE_RANGE','ERROR_MESSAGE']

#Create an empty CSV for monthly results    
with open(f"{output_path}/{name_of_output_file}_monthly.csv","w") as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(header)

#Create csv to save error output
with open(f"{output_path}/{name_of_output_file}_error_log.csv","w") as outcsv: 
    writer = csv.writer(outcsv)
    writer.writerow(error_header)  

## 10. Run tabulation script


In [81]:
#save batch/run information
run_start = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
count_of_sa2_complete = 0
count_of_sa2_error = 0
count_of_sa2_all = 0
sa2_min_size = 500_0000
sa2_max_size = 0
batch_start_information = pd.DataFrame([[name_of_output_file],[run_notes],[initials],[run_start]]).T
#append batch information to csv in order to record the start of the run
batch_start_information.to_csv(f"{output_path}/tabulate_FC_run_log.csv",mode='a',header=False) 

#run the tabulation script for all of the tiled SA2 polygons
for geometry, properties in shapes:
    count_of_sa2_all += 1
    
    #save information from shapefile
    sa2_id = str(properties['SA2_MAIN16'])
    sa2_name = str(properties['SA2_NAME16'])
    sa2_size = str(properties['AREASQKM16'])
    albers_id = str(properties['label'])
    print(f"SA2 ID: {sa2_id}, Albers tile: {albers_id}, SA2 size: {sa2_size}km^2, time: {time_range}")
        
    #loop through SA2 size and save the smallest and largest SA2 values
    if float(sa2_min_size) > float(sa2_size):
        sa2_min_size = sa2_size
    if float(sa2_max_size) < float(sa2_size):
        sa2_max_size = sa2_size
    
    #process fc tabulation
    try:
        process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, name_of_output_file, time_range)
        count_of_sa2_complete += 1      
    #if there are errors, save information to csv to record the name of SA2 and type of error
    except Exception as e:
            print(f"Could not process {sa2_id}: {e}")
            error_information = pd.DataFrame([[name_of_output_file],[sa2_id],[time_range], [e]]).T
            error_information.to_csv(f"{output_path}/{name_of_output_file}_error_log.csv",mode='a',header=False)
            count_of_sa2_error += 1
            
            
#save batch/run information
run_end = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
run_seconds = (datetime.strptime(run_end,'%Y-%m-%d %H:%M:%S') - datetime.strptime(run_start,'%Y-%m-%d %H:%M:%S')).total_seconds()

#save run/batch information to csv
batch_information = pd.DataFrame([[name_of_output_file],[run_notes],[initials],
                                  [run_start],[run_end],[run_seconds],[sa2_max_size],
                                  [sa2_min_size],[time_range],[count_of_sa2_all],
                                  [count_of_sa2_complete], [count_of_sa2_error],]).T
#output batch information to record the end of the run
batch_information.to_csv(f"{output_path}/tabulate_FC_run_log.csv",mode='a',header=False)

SA2 ID: 702051066, Albers tile: 2,-15, SA2 size: 92275.0274km^2, time: ('2015', '2017')
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07
2017-08
2017-09
2017-10
2017-11
2017-12
All data loaded, save to csv
SA2 702051066 done
SA2 ID: 702051066, Albers tile: 3,-15, SA2 size: 92275.0274km^2, time: ('2015', '2017')
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08
2015-09
2015-10
2015-11
2015-12
2016-01
2016-02
2016-03
2016-04
2016-05
2016-06
2016-07
2016-08
2016-09
2016-10
2016-11
2016-12
2017-01
2017-02
2017-03
2017-04
2017-05
2017-06
2017-07
2017-08
2017-09
2017-10
2017-11
2017-12
All data loaded, save to csv
SA2 702051066 done
SA2 ID: 702051066, Albers tile: 4,-15, SA2 size: 92275.0274km^2, time: ('2015', '2017')
2015-01
2015-02
2015-03
2015-04
2015-05
2015-06
2015-07
2015-08


## 11. Combine results for tiled SA2

In [82]:
#read area results of previous workflow and combine tiled results for each SA2

#read csv results and format into csv
df_in = pd.read_csv(f"{output_path}/{name_of_output_file}_monthly.csv", header=0)
df_in['DATE'] = pd.to_datetime(df_in['DATE'])
df = df_in[['DATE','SA2_MAIN16', 'SA2_NAME16','ALBERS3577_ID', 'AREASQKM16',
            'PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
           'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577',
            'FC_AREA_SQKM_ALBERS3577','AVERAGE_OBSERVATION_COUNT','SA2_TILE_PIXEL_COUNT']]

#group data to combine tiled results into whole of sa2 results
df = df.groupby(['SA2_MAIN16','DATE','SA2_NAME16','AREASQKM16']).sum()

#calculate the percentage of each FC componenet in relation to the whole size of SA2
df['PV_PERCENTAGE']= df.PV_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100
df['NPV_PERCENTAGE']= df.NPV_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100
df['BS_PERCENTAGE']= df.BS_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100
df['WOFL_PERCENTAGE']= df.WOFL_AREA_SQKM_ALBERS3577/df.FC_AREA_SQKM_ALBERS3577 *100

#calculate the percentage of SA2 that isn't classified as fc components (e.g. cloud)
df['UNCLASSIFIED_PERCENTAGE']= 100 - (df.PV_PERCENTAGE + df.NPV_PERCENTAGE + df.BS_PERCENTAGE + df.WOFL_PERCENTAGE)
unclass_area = df['UNCLASSIFIED_PERCENTAGE'] * (df['FC_AREA_SQKM_ALBERS3577']/100)
#reformat results
df.insert(7,'UNCLASSIFIED_AREA_SQKM_ALBERS3577',unclass_area)
df = df.reset_index(level=['DATE','AREASQKM16','SA2_NAME16'])

#calculate the "no data" quality metric
no_data_qm = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).count()
no_data_qm = no_data_qm.iloc[:,:1]
no_data_qm.rename(columns={'DATE':'MONTHS_WITH_NO_DATA'},inplace=True)
no_data_qm = 12 - no_data_qm

#remove all months that contain >10% unclassified
df = df.where(df.UNCLASSIFIED_PERCENTAGE <= 10).dropna()

#take a mean of all months within each year
annual_df = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).mean()

#calculate "low data" quality metric
low_data_qm = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).count()
low_data_qm = low_data_qm.iloc[:,:1]
low_data_qm.rename(columns={'DATE':'MONTHS_WITH_LOW_DATA'},inplace=True)
low_data_qm = 12 - low_data_qm

#Concetenate fc, and quality metrics together
annual_df_with_qm = pd.concat([annual_df,low_data_qm, no_data_qm],axis=1,join='inner')

#calculate "average observation quality metric"
ave_obs_qm = df.groupby([df.SA2_NAME16, df.DATE.dt.year]).sum()['AVERAGE_OBSERVATION_COUNT'] 
ave_obs_qm = ave_obs_qm/(annual_df_with_qm['SA2_TILE_PIXEL_COUNT']/100)

#reformat and add the average observation count metric
annual_df_with_qm.pop('AVERAGE_OBSERVATION_COUNT')
annual_df_with_qm ['AVERAGE_OBSERVATION_COUNT'] = ave_obs_qm
annual_df_with_qm.pop('SA2_TILE_PIXEL_COUNT')

#save annual results as csv
annual_df_with_qm.to_csv(f'{output_path}/{name_of_output_file}_annual.csv')

In [83]:
#Check annual results
annual_df_with_qm

AREASQKM16  PV_AREA_SQKM_ALBERS3577  \
SA2_NAME16 DATE                                        
Gulf       2015  92275.0274             24080.518099   
           2016  92275.0274             28516.000096   
           2017  92275.0274             30991.714434   

                 NPV_AREA_SQKM_ALBERS3577  BS_AREA_SQKM_ALBERS3577  \
SA2_NAME16 DATE                                                      
Gulf       2015              47414.373113             19199.927020   
           2016              45392.308811             16061.347635   
           2017              45631.432425             13103.414602   

                 WOFL_AREA_SQKM_ALBERS3577  FC_AREA_SQKM_ALBERS3577  \
SA2_NAME16 DATE                                                       
Gulf       2015                 328.338511             92274.991875   
           2016                 469.227145             92274.991875   
           2017                 540.366143             92274.991875   

                 UNCLASSIFIED_AREA_SQKM_ALBERS3577  PV_PERCENTAGE  \
SA2_NAME16 DATE                                                     
Gulf       2015                        1251.835131      26.096473   
           2016                        1836.108189      30.903281   
           2017                        2008.064270      33.586255   

                 NPV_PERCENTAGE  BS_PERCENTAGE  WOFL_PERCENTAGE  \
SA2_NAME16 DATE                                                   
Gulf       2015       51.383774      20.807292         0.355826   
           2016       49.192428      17.405959         0.508510   
           2017       49.451570      14.200396         0.585604   

                 UNCLASSIFIED_PERCENTAGE  MONTHS_WITH_LOW_DATA  \
SA2_NAME16 DATE                                                  
Gulf       2015                 1.356635                     3   
           2016                 1.989822                     1   
           2017                 2.176174                     2   

                 MONTHS_WITH_NO_DATA  AVERAGE_OBSERVATION_COUNT  
SA2_NAME16 DATE                                                  
Gulf       2015                    0                  32.450554  
           2016                    0                  30.970245  
           2017                    0                  33.602095